<center> <h3> Final Project | ANLY-580 </h3>
<h4>  Georgetown University | Fall 2018</h4>
<center>

***

#### Prep

In [155]:
# set working directory
import os
import sys
project_root = '/Users/kgedney/Documents/georgetown/anly580/wine_project'
os.chdir(project_root)

In [146]:
# install packages
import requests
import pandas as pd
from bs4 import BeautifulSoup

#### Set up scrape

This code scrapes the Wine Cellar Website.
- parses the url
- gets country and region
- gets all bottles for each type

In [147]:
# helper functions
def get_soup(url):
    x = requests.get(url).text
    return BeautifulSoup(x, 'lxml')

In [148]:
url = 'https://www.thewinecellarinsider.com/wine-tasting-regions?country=France'

In [150]:
from urllib.parse import urlparse, parse_qs
x = 'https://www.thewinecellarinsider.com/wine-tasting-regions?country=France&region=Bordeaux'

def parse_region_url(x):
    x = urlparse(x)
    q = parse_qs(x.query)
    return {
        'country' : q['country'][0],
        'region'  : q['region'][0],
    }


In [151]:
def get_regions():
    index_url = 'https://www.thewinecellarinsider.com/tasting-notes'
    soup = get_soup(index_url)
    links = soup.find_all('td')
    links = [link.find('a') for link in links if link is not None]
    links = [link['href'] for link in links if link is not None]
    links = [link for link in links if 'region=' in link]
    links = [base_url + link for link in links]
    return links

region_urls = get_regions()

In [152]:
def get_producer_links(soup):
    base_url = 'https://www.thewinecellarinsider.com'
    links = soup.find_all('li')
    links = [link.find('h4') for link in links]
    links = [link.find('a') for link in links if link is not None]
    links = [link['href'] for link in links if link is not None]
    links = [base_url + link for link in links]
    return links

def process_view_row(x):
    vals = [td.text.strip() for td in x.find_all('td')]
    return dict(zip(['year', 'bottle', 'rating'], vals))

def process_fold_row(x):
    reviews = x.find_all(itemprop='reviews')
    texts   = [review.find('p').text.strip() for review in reviews]
    return texts

def process_producer_page(soup):
    view_rows = soup.find_all('tr', {"class" : 'view'})
    view_rows = [process_view_row(row) for row in view_rows]

    fold_rows = soup.find_all('tr', {"class" : 'fold'})
    fold_rows = [process_fold_row(row) for row in fold_rows]
    
    assert len(view_rows) == len(fold_rows)
    out = []
    for view, fold in zip(view_rows, fold_rows):
        for note_id, note in enumerate(fold):
            out.append({
                "year"     : view['year'],
                "bottle"   : view['bottle'],
                "rating"   : view['rating'],
                "note"     : note,
                "note_id"  : note_id
            }) 
    
    return out

#### Run scrape (takes a few hours)

In [156]:
from tqdm import tqdm
from time import sleep

all_results = []
for region_url in region_urls:
    try:
        print(region_url)
        region_params  = parse_region_url(region_url)
        region_soup    = get_soup(region_url)
        producer_links = get_producer_links(region_soup)
        
        for producer_link in tqdm(producer_links):
            producer_soup = get_soup(producer_link)
            producer_info = process_producer_page(producer_soup)
            for p_info in producer_info:
                p_info.update(region_params)
                all_results.append(p_info)
    except KeyboardInterrupt:
        raise
    except:
        print('error at %s' % region_url, file=sys.stderr)

    sleep(1)

https://www.thewinecellarinsider.com/wine-tasting-regions?country=France&region=Bordeaux




  0%|          | 0/694 [00:00<?, ?it/s]

  0%|          | 1/694 [00:02<31:59,  2.77s/it]

  0%|          | 2/694 [00:05<31:08,  2.70s/it]

  0%|          | 3/694 [00:07<30:11,  2.62s/it]

  1%|          | 4/694 [00:10<29:40,  2.58s/it]

  1%|          | 5/694 [00:12<29:16,  2.55s/it]

  1%|          | 6/694 [00:15<29:31,  2.57s/it]

  1%|          | 7/694 [00:17<29:27,  2.57s/it]

  1%|          | 8/694 [00:20<28:38,  2.50s/it]

  1%|▏         | 9/694 [00:22<29:19,  2.57s/it]

  1%|▏         | 10/694 [00:25<28:41,  2.52s/it]

  2%|▏         | 11/694 [00:27<28:55,  2.54s/it]

  2%|▏         | 12/694 [00:30<28:10,  2.48s/it]

  2%|▏         | 13/694 [00:33<30:44,  2.71s/it]

  2%|▏         | 14/694 [00:36<31:21,  2.77s/it]

  2%|▏         | 15/694 [00:39<30:54,  2.73s/it]

  2%|▏         | 16/694 [00:41<30:02,  2.66s/it]

  2%|▏         | 17/694 [00:44<30:16,  2.68s/it]

  3%|▎         | 18/694 [00:47<30:30,  2.71s/it]

  3%|▎         | 19/694 [00:49<29:37,  2.63s/it]

  3%|▎         |

 41%|████▏     | 288/694 [12:55<18:33,  2.74s/it]

 42%|████▏     | 289/694 [12:58<18:36,  2.76s/it]

 42%|████▏     | 290/694 [13:01<19:11,  2.85s/it]

 42%|████▏     | 291/694 [13:04<19:38,  2.93s/it]

 42%|████▏     | 292/694 [13:07<18:58,  2.83s/it]

 42%|████▏     | 293/694 [13:11<21:22,  3.20s/it]

 42%|████▏     | 294/694 [13:14<20:24,  3.06s/it]

 43%|████▎     | 295/694 [13:17<20:20,  3.06s/it]

 43%|████▎     | 296/694 [13:20<19:51,  2.99s/it]

 43%|████▎     | 297/694 [13:22<19:08,  2.89s/it]

 43%|████▎     | 298/694 [13:25<19:28,  2.95s/it]

 43%|████▎     | 299/694 [13:28<19:24,  2.95s/it]

 43%|████▎     | 300/694 [13:31<19:26,  2.96s/it]

 43%|████▎     | 301/694 [13:34<18:24,  2.81s/it]

 44%|████▎     | 302/694 [13:36<17:41,  2.71s/it]

 44%|████▎     | 303/694 [13:39<17:32,  2.69s/it]

 44%|████▍     | 304/694 [13:41<17:03,  2.62s/it]

 44%|████▍     | 305/694 [13:44<17:03,  2.63s/it]

 44%|████▍     | 306/694 [13:47<17:05,  2.64s/it]

 44%|████▍     | 307/694 [13:49

 83%|████████▎ | 574/694 [25:51<05:34,  2.79s/it]

 83%|████████▎ | 575/694 [25:53<05:14,  2.65s/it]

 83%|████████▎ | 576/694 [25:56<05:21,  2.72s/it]

 83%|████████▎ | 577/694 [25:59<05:21,  2.75s/it]

 83%|████████▎ | 578/694 [26:01<05:15,  2.72s/it]

 83%|████████▎ | 579/694 [26:04<05:16,  2.76s/it]

 84%|████████▎ | 580/694 [26:06<04:59,  2.62s/it]

 84%|████████▎ | 581/694 [26:09<04:41,  2.49s/it]

 84%|████████▍ | 582/694 [26:12<04:59,  2.68s/it]

 84%|████████▍ | 583/694 [26:14<04:54,  2.65s/it]

 84%|████████▍ | 584/694 [26:17<04:47,  2.61s/it]

 84%|████████▍ | 585/694 [26:19<04:39,  2.56s/it]

 84%|████████▍ | 586/694 [26:22<04:39,  2.59s/it]

 85%|████████▍ | 587/694 [26:24<04:32,  2.55s/it]

 85%|████████▍ | 588/694 [26:27<04:35,  2.60s/it]

 85%|████████▍ | 589/694 [26:30<04:33,  2.60s/it]

 85%|████████▌ | 590/694 [26:32<04:16,  2.47s/it]

 85%|████████▌ | 591/694 [26:35<04:35,  2.68s/it]

 85%|████████▌ | 592/694 [26:38<04:52,  2.87s/it]

 85%|████████▌ | 593/694 [26:41

https://www.thewinecellarinsider.com/wine-tasting-regions?country=France&region=Rh%F4ne


...



  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.26s/it]



https://www.thewinecellarinsider.com/wine-tasting-regions?country=Lebanon&region=Bekaa%20Valley




  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.90s/it]



https://www.thewinecellarinsider.com/wine-tasting-regions?country=Ukraine&region=Crimea




  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.07s/it]



In [159]:
# save results
print(len(all_results))
df = pd.DataFrame(all_results)
df.to_csv('data/cellarinsider_raw.csv')

,bottle,country,note,note_id,rating,region,year
0,Domaine de l'A (Castillon Côtes de Bordeaux),France,"Showing a respectable color, the nose kicks of...",0,91,Bordeaux,2017
1,Domaine de l'A (Castillon Côtes de Bordeaux),France,"Flowers, spice, licorice and juicy plums domin...",0,93,Bordeaux,2016
2,Domaine de l'A (Castillon Côtes de Bordeaux),France,"Distinctly floral with sweet, fresh, dark red ...",0,92,Bordeaux,2015
3,Domaine de l'A (Castillon Côtes de Bordeaux),France,"In the nose, you find black raspberries, with ...",0,90,Bordeaux,2014
4,Domaine de l'A (Castillon Côtes de Bordeaux),France,Owned by renowned consultant Stephane Derenonc...,0,90,Bordeaux,2012


In [162]:
df.shape

(17713, 7)